In [ ]:
pip install deepsparse pandas transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.9 MB/s eta 0:00:00


In [ ]:
from deepsparse import compile_model

stub = "zoo:roberta-base-conll2003_wikipedia_bookcorpus-pruned85"
engine = compile_model(stub, batch_size=1)
print(engine)

Combining Chunks: 100%|██████████| 189M/189M [00:00<00:00, 641MB/s]


deepsparse.engine.Engine:
	onnx_file_path: /root/.cache/sparsezoo/neuralmagic/roberta-base-conll2003_wikipedia_bookcorpus-pruned85/deployment/model.onnx
	batch_size: 1
	num_cores: 48
	num_streams: 1
	scheduler: Scheduler.default
	fraction_of_supported_ops: 0.9825
	cpu_avx_type: avx512
	cpu_vnni: False


In [ ]:
import pandas as pd
import pickle
import numpy as np
import torch
from deepsparse import Engine
from transformers import AutoTokenizer

In [ ]:
# Load the news dataset TSV file
df = pd.read_csv("news_min.tsv", sep='\t')

# Load the tokenizer (RoBERTa-based model)
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
df.head(10)

,News ID,Category,Topic,Headline,News body
0,N10000,sports,soccer,Predicting Atlanta United's lineup against Col...,"Only FIVE internationals allowed, count em, FI..."
1,N10001,news,newspolitics,Mitch McConnell: DC statehood push is 'full bo...,WASHINGTON -- Senate Majority Leader Mitch McC...
2,N10002,news,newsus,Home In North Highlands Damaged By Fire,NORTH HIGHLANDS (CBS13) Fire damaged a home ...
3,N10003,news,newspolitics,Meghan McCain blames 'liberal media' and 'thir...,Meghan McCain is speaking out after a journali...
4,N10004,news,newsworld,Today in History: Aug 1,"1714: George I becomes King Georg Ludwig, Elec..."
5,N10005,sports,football_nfl,Odell Beckham Jr New Custom Rolls Royce Cullinan,Odell Beckham Jr New Custom Rolls Royce Cullin...
6,N10006,autos,autosclassics,This Attention-Grabbing Chevrolet Malibu Packs...,This muscle car has the power to justify its d...
7,N10007,news,newspolitics,GOP senators urge Trump to reject Iran's 'nucl...,Three GOP senators this week urged President T...
8,N10008,foodanddrink,newstrends,5 great Mexican restaurants in Lebanon County,The latest from the Richland Borough Council M...
9,N10009,sports,football_nfl,NFL officials union head expects a lot of pres...,The NFL has formalized the language of the rul...


In [ ]:
def get_embeddings(text, model=engine):
    """
    Generate sentence embeddings using DeepSparse engine and tokenizer.

    Args:
        text (str): The input text for which to compute embeddings.
        model (Engine): The DeepSparse model engine (default is the loaded engine).

    Returns:
        torch.Tensor: Sentence embedding vector.
    """
    text = str(text)
    # Tokenize the input text with max_length=128 to match model's expected input size
    tokens = tokenizer(text, return_tensors='np', padding='max_length', truncation=True, max_length=128)

    # Convert input_ids and attention_mask to numpy arrays with int64 dtype
    input_ids = np.array(tokens['input_ids'], dtype=np.int64)
    attention_mask = np.array(tokens['attention_mask'], dtype=np.int64)

    # Pass both input_ids and attention_mask to the model
    embeddings = model([input_ids,attention_mask])

    # Convert the embedding result to a torch tensor (first element of the first batch)
    embedding = torch.tensor(embeddings[0][0])

    return embedding


In [ ]:
from tqdm import tqdm
import os

In [ ]:
checkpoint_file = 'news_embeddings_checkpoint.pkl'
checkpoint_freq = 5000  # Save checkpoint every 10,000 rows

In [ ]:
if os.path.exists(checkpoint_file):
    # Load the checkpoint and get the last processed index
    with open(checkpoint_file, 'rb') as f:
        news_embeddings_dict, last_processed_index = pickle.load(f)
    print(f"Resuming from checkpoint. Last processed row: {last_processed_index}")
else:
    # Start from scratch if no checkpoint is found
    news_embeddings_dict = {}
    last_processed_index = -1

# Iterate through each row in the dataset
for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing news articles"):
    news_id = row['News ID']    # Get the News ID
    news_body = row['News body']  # Get the News body text

    # Get the embeddings for the News body text
    embedding = get_embeddings(news_body)

    # Store the embeddings in the dictionary with news_id as the key
    news_embeddings_dict[news_id] = embedding

    if index > 0 and index % checkpoint_freq == 0:
        with open(checkpoint_file, 'wb') as f:
            pickle.dump((news_embeddings_dict, index), f)
        print(f"Checkpoint saved at row: {index}")

# Save the embeddings dictionary as a .pkl file
with open('news_embeddings.pkl', 'wb') as f:
    pickle.dump(news_embeddings_dict, f)

print("Embeddings have been saved to 'news_embeddings.pkl'.")

Processing news articles:   0%|          | 507/113762 [00:09<35:29, 53.19it/s]

Checkpoint saved at row: 500


Processing news articles:   1%|          | 1010/113762 [00:20<33:23, 56.29it/s]

Checkpoint saved at row: 1000


Processing news articles:   1%|▏         | 1508/113762 [00:31<53:06, 35.23it/s]  

...

Checkpoint saved at row: 113000


Processing news articles: 100%|█████████▉| 113511/113762 [58:51<01:17,  3.22it/s]

Checkpoint saved at row: 113500


Processing news articles: 100%|██████████| 113762/113762 [58:58<00:00, 32.15it/s]


Embeddings have been saved to 'news_embeddings.pkl'.


In [ ]:
pickle_file_path = '/content/news_embeddings.pkl'

with open(pickle_file_path, 'rb') as f:
    # The pickle file contains only the news_embeddings_dict
    news_embeddings_dict = pickle.load(f)
    # Remove last_processed_index as it's not stored in the file
    # Initialize or get it from elsewhere if needed
    # For example, you could reset it to -1 or get it from the checkpoint file if available
    last_processed_index = -1

# Display the contents of the pickle file
print(f"Last processed index: {last_processed_index}")  # May need to adjust depending on how you set last_processed_index
print("First 10 Embeddings from the dictionary:\n")

# Print out the first 10 entries
for news_id, embedding in list(news_embeddings_dict.items())[:10]:
    print(f"News ID: {news_id}")
    print(f"Embedding (shape: {embedding.shape}): {embedding}\n")

/usr/local/lib/python3.10/dist-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


Last processed index: -1
First 10 Embeddings from the dictionary:

News ID: N10000
Embedding (shape: torch.Size([128, 9])): tensor([[ 1.0791, -0.0520, -0.6545,  ..., -0.8640,  0.2703, -0.8104],
        [10.2101, -1.1307, -2.1981,  ..., -2.1493, -1.5064, -2.2739],
        [ 6.6266, -0.4424, -3.0224,  ..., -2.8820,  1.3735, -2.1363],
        ...,
        [10.3190, -1.6721, -2.4339,  ..., -2.0109, -1.7192, -1.7840],
        [ 1.1526,  1.0283, -2.8732,  ..., -3.0361,  0.5396, -2.9618],
        [ 1.0785, -0.0523, -0.6550,  ..., -0.8642,  0.2719, -0.8099]])

News ID: N10001
Embedding (shape: torch.Size([128, 9])): tensor([[ 1.0557, -0.0509, -0.6470,  ..., -0.8223,  0.2251, -0.8401],
        [-0.0704,  0.1551, -2.5323,  ..., -0.2067, -1.5623, -3.3790],
        [10.2061, -1.5499, -2.3800,  ..., -1.8061, -1.8805, -2.1766],
        ...,
        [10.5437, -1.2833, -2.3469,  ..., -2.1078, -1.5302, -2.4848],
        [10.6229, -1.5746, -2.2555,  ..., -1.8906, -1.7168, -2.2884],
        [ 1.0545, -0.

In [ ]:
def distance(embedding1, embedding2):
    # Calculate the squared differences and take the mean
    squared_diff = np.square(embedding1 - embedding2)
    return np.mean(squared_diff)

In [ ]:
def divergence(Dt1, Dt2, Ut1, Ut2):
    # Calculate the distances using the mean squared distance function
    document_dist1 = distance(Dt1, Ut1)
    document_dist2 = distance(Dt2, Ut2)
    doc_dist_t1_t2 = distance(Dt1, Dt2)
    user_dist_t1_t2 = distance(Ut1, Ut2)

    # Compute the divergence as per the given formula
    divergence_value = (document_dist1 - document_dist2) / (doc_dist_t1_t2 - user_dist_t1_t2)

    return divergence_value

In [ ]:
with open(pickle_file_path, 'rb') as f:
    news_embeddings_data = pickle.load(f)

if isinstance(news_embeddings_data, dict):
    news_embeddings = news_embeddings_data
else:
    # If it's not a dictionary, process it accordingly (this depends on the data format)
    # For instance, if it contains a list of tuples, convert to a dictionary
    # Example: news_embeddings_data = [(news_id, embedding), (news_id, embedding), ...]
    news_embeddings = {item[0]: item[1] for item in news_embeddings_data}

# Now news_embeddings is a dictionary mapping news IDs to embeddings
print(news_embeddings)

Buffered data was truncated after reaching the output size limit.

In [ ]:
dataset = pd.read_csv('/content/personalized_test.tsv', sep='\t', on_bad_lines='skip')

dataset.head(10)

In [ ]:
# Iterate over each user trajectory
overall_divergence = 0
for index, row in dataset.iterrows():
    user_id = row['userid']
    clicked_news_id = row['clicknewsID']
    news_ids = row['posnewID']
    summaries = row['rewrite_titles']

    # Loop over each timestamp in the trajectory, assuming the embedding of user at t1 is summary embedding at t1
    for t1 in range(len(news_ids) - 1):
        # Get embeddings for documents
        Dt1 = news_embeddings[news_ids[t1]]
        Dt2 = news_embeddings[news_ids[t1 + 1]]

    '''
    for t1 in range(len(clicked_news_id) - 1):
        # Get embeddings for documents
        Dt1 = news_embeddings[clicked_news_id[t1]]
        Dt2 = news_embeddings[clicked_news_id[t1 + 1]]
    '''

        # Get user embeddings at time t1 and t2 (assume user embedding is the same as the summary embedding at that time)
        Ut1 = get_embeddings(summaries[t1])
        Ut2 = get_embeddings[summaries[t1 + 1]]

        # Calculate divergence using the given formula
        document_divergence = (divergence(Dt1, Ut1) - divergence(Dt2, Ut2)) / (divergence(Dt1, Dt2) - divergence(Ut1, Ut2))
        overall_divergence += document_divergence

# Normalize the overall divergence (by the number of trajectories, for example)
overall_divergence = 0.5 * overall_divergence / len(dataset)

# Output the final overall divergence
print("Overall divergence in the dataset:", overall_divergence)